In [38]:
def readFeaturesWithSums(jsonTest,test):
    res=[]
    res.append(['match_id','r_xp_sum','d_xp_sum','r_gold_sum','d_gold_sum','r_lh_sum',
                'd_lh_sum','d_killers_count','r_killers_count'])
    with bz2.BZ2File(jsonTest) as matches_file:
        for line in matches_file:
            match = json.loads(line)
            r_xp_sum=0
            d_xp_sum=0
            r_gold_sum=0
            d_gold_sum=0
            r_lh_sum=0
            d_lh_sum=0
            r_killers_count=len(match['players'][0]['kills_log'])
            +len(match['players'][1]['kills_log'])
            +len(match['players'][2]['kills_log'])
            +len(match['players'][3]['kills_log'])
            +len(match['players'][4]['kills_log'])
            d_killers_count=len(match['players'][5]['kills_log'])
            +len(match['players'][6]['kills_log'])
            +len(match['players'][7]['kills_log'])
            +len(match['players'][8]['kills_log'])
            +len(match['players'][9]['kills_log'])
            
            for i in range(len(match['times'])):
                if(match['times'][i]<=5*60):
                    r_xp_sum=match['players'][0]['xp_t'][i]+match['players'][1]['xp_t'][i]+match['players'][2]['xp_t'][i]+match['players'][3]['xp_t'][i]+match['players'][4]['xp_t'][i]
        
                    d_xp_sum=match['players'][5]['xp_t'][i]\
                    +match['players'][6]['xp_t'][i]\
                    +match['players'][7]['xp_t'][i]\
                    +match['players'][8]['xp_t'][i]\
                    +match['players'][9]['xp_t'][i]
        
                    r_gold_sum=match['players'][0]['gold_t'][i]\
                    +match['players'][1]['gold_t'][i]\
                    +match['players'][2]['gold_t'][i]\
                    +match['players'][3]['gold_t'][i]\
                    +match['players'][4]['gold_t'][i]
        
                    d_gold_sum=match['players'][5]['gold_t'][i]\
                    +match['players'][6]['gold_t'][i]\
                    +match['players'][7]['gold_t'][i]\
                    +match['players'][8]['gold_t'][i]\
                    +match['players'][9]['gold_t'][i]
     
                    r_lh_sum=match['players'][0]['lh_t'][i]\
                    +match['players'][1]['lh_t'][i]\
                    +match['players'][2]['lh_t'][i]\
                    +match['players'][3]['lh_t'][i]\
                    +match['players'][4]['lh_t'][i]
        
                    d_lh_sum=match['players'][5]['lh_t'][i]\
                    +match['players'][6]['lh_t'][i]\
                    +match['players'][7]['lh_t'][i]\
                    +match['players'][8]['lh_t'][i]\
                    +match['players'][9]['lh_t'][i]
            res.append([match['match_id'],r_xp_sum,d_xp_sum,r_gold_sum,d_gold_sum,
                        r_lh_sum,d_lh_sum,d_killers_count,d_killers_count])
    import csv
    with open('tmp.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerows(res)
    features = pandas.read_csv(test)
    s = pandas.read_csv('./tmp.csv', index_col='match_id')
    result = pandas.merge(features, s, left_on='match_id', right_index=True, how='left', sort=False)
    return result

In [ ]:
'''
#test=readFeaturesWithSums('matches_test.jsonlines.bz2','features_test.csv')
#print('test')
features=readFeaturesWithSums('matches.jsonlines.bz2','features.csv')
#features=pandas.read_csv('features.csv')
#test=pandas.read_csv('features_test.csv')
print('features')
features.fillna(9999999,inplace=True)
test.fillna(9999999,inplace=True)
y=features['radiant_win']
features=features.drop(['duration','radiant_win','tower_status_radiant',
               'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)

'''
clf=GradientBoostingClassifier(n_estimators=180, random_state=42,max_depth=7)
clf.fit(features,y)
print('fit, yea')
pred=clf.predict_proba(test)[:,1]
match_id=pandas.read_csv('./features_test.csv')['match_id']
total=[]
total.append(["match_id","radiant_win"])
for i in range(0,len(match_id)):
    total.append([match_id[i],pred[i]])
import csv
with open('res.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(total)
('done')

In [40]:
test.loc[test['match_id'] == 112916]


,match_id,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,...,dire_ward_sentry_count,dire_first_ward_time,r_xp_sum,d_xp_sum,r_gold_sum,d_gold_sum,r_lh_sum,d_lh_sum,d_killers_count,r_killers_count
16949,112916,1450023914,1,8,4,1431,1537,22,0,1,...,3,-71.0,5481,5789,5345,5496,48,49,2,2


In [44]:
pred=clf.predict_proba(test)[:,1]
match_id=pandas.read_csv('./features_test.csv')['match_id']
total=[]
total.append(["match_id","radiant_win"])
for i in range(0,len(match_id)):
    total.append([match_id[i],pred[i]])
import csv
with open('res.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(total)

In [18]:
#Repeat good models
d_features=features
d_y=y
d_test=test
d_clf=GradientBoostingClassifier(n_estimators=180, random_state=42,max_depth=7)
d_clf.fit(d_features,d_y)
d_pred=d_clf.predict_proba(d_test)[:,1]
d_match_id=pandas.read_csv('./features_test.csv')['match_id']
d_total=[]
d_total.append(["match_id","radiant_win"])
for i in range(0,len(d_match_id)):
    d_total.append([d_match_id[i],d_pred[i]])
import csv
with open('d_res.csv', 'w', newline='') as csvfile:
    d_spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    d_spamwriter.writerows(d_total)

In [16]:
features.r_gold_sum.head

<bound method NDFrame.head of 0        10358
1         9696
2        10942
3         7924
4         9099
5         9315
6        10765
7         8964
8         8515
9         8116
10       12537
11        8048
12       11625
13       10599
14        9285
15       12036
16       12658
17        9142
18        8623
19        9898
20       10237
21       12455
22       13464
23       11208
24       10533
25       10900
26       11511
27       11613
28        8219
29        9049
         ...  
97200     9517
97201    10231
97202    11884
97203    13157
97204    11477
97205     9837
97206    11613
97207     9918
97208     9939
97209     8672
97210     9495
97211    11857
97212    12778
97213    10891
97214    12404
97215    12962
97216    11147
97217     9029
97218     9172
97219    11899
97220    12590
97221    10743
97222    15810
97223    10562
97224     9848
97225    10537
97226    10374
97227    10179
97228    10246
97229    12524
Name: r_gold_sum, Length: 97230, dtype: int64>